In [0]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from scipy.io import loadmat
import h5py
from scipy import signal
from math import pi
from sklearn.preprocessing import MinMaxScaler,StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
# directory (dependent on what comp being used)
data_dir='./gdrive/My Drive/BMME890_MachineLearning/ProjectData'
#data_dir ='C:/Users/nmrubin/Desktop/BMME890MachineLearningProject/GoogleDrive/ProjectData'

#import neutral position emg data
#loadmat can't work with matlab v 7.3, use h5py

#ordered by subjects 1-8
#Subjects= ['LV','KH','XH','HS','RC','NR','YZ','AM']
Subjects=['LV','KH','HS','RC','NR','YZ','AM']
EMG_dir = data_dir + '/EMGData/'
Force_dir = data_dir + '/ForceData/'
Spks_dir = data_dir + '/SpikeTrainsData/'


In [0]:
def rms(matrix,axis):
    
    return np.sqrt(np.mean(matrix**2,axis=axis))

def crosscorr(a,b):
    lag=np.correlate(a-np.mean(a),b-np.mean(b),mode='full')
    lag=np.argmax(lag)-len(b)
    return lag

def trim_MUdelay(delay,FR,force,finaltrim):        
    FR = np.roll(FR,shift = abs(delay))
    force=np.squeeze(force)
    trimmedForce=force[abs(delay):]
    trimmedFR = FR[abs(delay):]
    # print('Frc')
    # print(len(trimmedForce))
    # print('FR')
    # print(len(trimmedFR))
    trimmedForce=trimmedForce[0:finaltrim]
    return trimmedFR,trimmedForce


def rms_top_channels(matrix,topN):
    rmsmatrix=rms(matrix,1)
    channelInd=np.argpartition(rmsmatrix,-topN)[-topN:] 
    topchannels=matrix[channelInd,:]
    return topchannels

def rolling_windows(signal,window,step,frequency):
        
        #check which dimension is time/longer
        dim = np.argmax(np.shape(signal))
        
        windowSize=round(window*frequency)  
        stepSize=round(step*frequency)
        windows=[]
        windtimes = []
        for start in np.arange(0,signal.shape[dim]-windowSize+1,stepSize):
            
            #track window times
            if start == 0: #start at half a window
                windtimes.append(window/2)
            else: #increment by one window step size
                windtimes.append(windtimes[-1] + step)
            
            end=start+windowSize
            
            if len(np.shape(signal)) == 1: #if column vector
                windows.append(np.array(signal[start:end]))
            else:
                if dim == 0:
                    windows.append(np.array(signal[start:end,:])) #go down rows
                else:
                    windows.append(np.array(signal[:,start:end])) #go down column
        
        return windows,windtimes

    #get indices of the windows needed for cross-validation
def crossvalidataprep(windtimes):
        windtimes = np.array(windtimes)
        crossvalsectind = []
        Crossvaltimes = [0,26,49,72,95,118,141] #seconds, timing cutoffs for each repetition (7 total, last 141-end)
        for i in range(len(Crossvaltimes)):
            if i == 0:
                crossvalsectind.append(0)
            else:
                tmp = windtimes - Crossvaltimes[i]
                crossvalsectind.append(np.argmin(np.abs(tmp)))          
        return crossvalsectind

def extrafeatures(RMSEMGwind):
        zc = []
        #ssc = []
        #std = []
        maxamp = []

        for window in RMSEMGwind:

            #zero crossings
            zc.append(len(np.ravel(np.diff(np.sign(window))).nonzero()[0]))

            #ssc
            #dev2=np.gradient(np.gradient(window))
            #ssc.append(len(np.ravel(np.diff(np.sign(dev2))).nonzero()[0]))

            #max amplitude
            maxamp.append(np.max(window))

            #std deviation
            #std.append(np.std(window))
            
            extrafeat = pd.DataFrame({'Zero-Cross': zc,'Maxamp': maxamp})
            
        return extrafeat

    #Calculate firing rate of Spike Train across windows (1603 windows)
def FRwind(SpkTrn,WinLength,StepLength,EMGFs):
    # SpkTrn = SpkTrnNNI
    # dim = np.argmax(np.shape(signal))
        emglength = np.shape(SpkTrn)[1]
        WinLength = 0.5 #seconds
        StepLength = 0.1
        WinNumSample = int(EMGFs*WinLength)
        StepNumSample = int(EMGFs*StepLength)
        TimeMax = emglength/EMGFs
        TimeWin = np.arange(WinNumSample/EMGFs,TimeMax,StepNumSample/EMGFs,dtype='float')
        timeemg = np.arange(1/EMGFs,(1+emglength)/EMGFs,1/EMGFs,dtype='float')

        FR = []

        MUs = np.shape(SpkTrn)[0]

        for i in range(1+np.size(TimeWin)):
            if i == 0: #first window
                windstartind = 0
                windstopind = np.argmin(abs(timeemg - WinLength))

            else: #move window forward
                windstartind = windstartind + StepNumSample + 1
                windstopind = windstopind + StepNumSample + 1

            if windstopind > emglength: #if next window goes past length of EMG, cut it off early
                windstopind = emglength
                WinLength = (windstopind - windstartind + 1)/EMGFs #last window time for new FR computation

            #compute firing rate in window
            FRtemp = np.zeros((1,MUs))
            for mu in range(MUs):
                FRtemp[0,mu] = np.count_nonzero(SpkTrn[mu,windstartind:windstopind])/WinLength

            FR.append((np.mean(FRtemp))) #get average firing rate

            if windstopind == emglength: #end of that was last window
                break
        return FR

def trim_delay(delay,data,rms,force,finaltrim):
    trimmedData=pd.DataFrame()
    for feature in data.columns:
        featdata=np.roll(data[feature],shift=abs(delay))
        featdata=featdata[abs(delay):]
        trimmedData[feature]=featdata
    
    rms = np.roll(rms,shift = abs(delay))
    force=np.squeeze(force)
    trimmedForce=force[abs(delay):]  
    trimmedRMS=rms[abs(delay):]
    trimmedData['rms']=trimmedRMS
    trimmedData=trimmedData.iloc[0:finaltrim]
    trimmedForce=trimmedForce[0:finaltrim]
    return trimmedData,trimmedForce


In [0]:
def get_subj_var(Force_dir,EMG_dir,Spks_dir,Subj):
    ForceFs = 1000

    with h5py.File(Force_dir + Subj + '_Forces.mat', 'r') as file:
        print('force')
        ForceNI = np.array(file['ForceNItrial'])
        ForceNItime = np.array(file['ForceNItime'])
        ForceNM = np.array(file['ForceNMtrial'])
        ForceNMtime = np.array(file['ForceNMtime'])
        
        ForcePI = np.array(file['ForcePItrial'])
        ForcePItime = np.array(file['ForcePItime'])
        ForcePM = np.array(file['ForcePMtrial'])
        ForcePMtime = np.array(file['ForcePMtime'])
        
        ForceSI = np.array(file['ForceSItrial'])
        ForceSItime = np.array(file['ForceSItime'])
        ForceSM = np.array(file['ForceSMtrial'])
        ForceSMtime = np.array(file['ForceSMtime'])

    #load emg data
    #careful, will take up ~4-8GB Memory

    #NPS = Neutral/Pronated/Supinated
    #IMRP = Index/Middle/Ring/Pinky

    #pick sample rate for appropriate subject
    if Subj == 'LV':
        EMGFs =  2.5621e+03
    else:
        EMGFs =  2.0497e+03

    with h5py.File(EMG_dir + Subj + '_EMGtrials.mat', 'r') as file:
        print('emg')
        EMGNI = np.array(file['EMGNI'])
        EMGNM = np.array(file['EMGNM'])
        
        EMGPI = np.array(file['EMGPI'])
        EMGPM = np.array(file['EMGPM'])
        
        EMGSI = np.array(file['EMGSI'])
        EMGSM = np.array(file['EMGSM'])
        
    #first repetition is skipped in case of user-adjustments at beginning of trial, cut off first 19 seconds of EMG data to match with force
    cutind = int(np.floor(19*EMGFs))
    EMGNI = EMGNI[:,cutind:]
    EMGNM = EMGNM[:,cutind:]

    EMGPI = EMGPI[:,cutind:]
    EMGPM = EMGPM[:,cutind:]

    EMGSI = EMGSI[:,cutind:]
    EMGSM = EMGSM[:,cutind:]

    #load spiketrain data

    #SpkTrn_PostureDecomposed_PostureforTrial_Finger
    #e.g. SpkTrnNSI = Neutral MUs, Supinated Trial, Index Finger
    #e.g. SpkTrnPPM = Pronated MUs, Pronated Trial, Middle Finger

    #compare Neutral MUs against MUs decomposed in Pronated/Supinated positions

    with h5py.File(Spks_dir + Subj + '_MUsyncscompiled.mat', 'r') as file:
        print('spk')
        SpkTrnNNI = np.array(file['SpkTrnNNI'])
        SpkTrnNNM = np.array(file['SpkTrnNNM'])
            
        SpkTrnNPI = np.array(file['SpkTrnNPI'])
        SpkTrnNPM = np.array(file['SpkTrnNPM'])
        
        SpkTrnNSI = np.array(file['SpkTrnNSI'])
        SpkTrnNSM = np.array(file['SpkTrnNSM'])
        
        SpkTrnPPI = np.array(file['SpkTrnPPI'])
        SpkTrnPPM = np.array(file['SpkTrnPPM'])
        
        SpkTrnSSI = np.array(file['SpkTrnSSI'])
        SpkTrnSSM = np.array(file['SpkTrnSSM'])

    """Note: After cutoff of EMG data it still has 1 extra index compared to SpkTrn, not sure why"""

    forcetime = np.size(ForceNI)/1000
    spktime = np.shape(SpkTrnNNM)[1]/EMGFs
    emgtime = np.shape(EMGPI)[1]/EMGFs

    

    #get mean rms values of top 85 channels
    NIrms=rms_top_channels(EMGNI,85)
    NMrms=rms_top_channels(EMGNM,85)

    PIrms=rms_top_channels(EMGPI,85)
    PMrms=rms_top_channels(EMGPM,85)

    SIrms=rms_top_channels(EMGSI,85)
    SMrms=rms_top_channels(EMGSM,85)

    
    #crossvalsectind is vector of 6 values
    #section 1 = crossvalind[0] thru crossvalidind[1]-1
    #section 7 = crossvalind[end] to last window in signal

    #feature generation, compute new features and combine into dataframe

    

    #get firing rate windows
    WinLength = .5
    StepLength = .1
    FRwindNNI = FRwind(SpkTrnNNI,WinLength,StepLength,EMGFs)
    FRwindNNM = FRwind(SpkTrnNNM,WinLength,StepLength,EMGFs)

    FRwindNSI = FRwind(SpkTrnNSI,WinLength,StepLength,EMGFs)
    FRwindNSM = FRwind(SpkTrnNSM,WinLength,StepLength,EMGFs)

    FRwindNPI = FRwind(SpkTrnNPI,WinLength,StepLength,EMGFs)
    FRwindNPM = FRwind(SpkTrnNPM,WinLength,StepLength,EMGFs)

    FRwindPPI = FRwind(SpkTrnNPI,WinLength,StepLength,EMGFs)
    FRwindPPM = FRwind(SpkTrnNPM,WinLength,StepLength,EMGFs)

    FRwindSSI = FRwind(SpkTrnSSI,WinLength,StepLength,EMGFs)
    FRwindSSM = FRwind(SpkTrnSSM,WinLength,StepLength,EMGFs)

    #Get emg windows
    EMGNIwind,EMGNIwindtimes = rolling_windows(NIrms,.5,.1,EMGFs)
    ForceNIwind,ForceNIwindtimes = rolling_windows(ForceNI,.5,.1,1000)
    EMGNMwind,EMGNMwindtimes = rolling_windows(NMrms,.5,.1,EMGFs)
    ForceNMwind,ForceNMwindtimes = rolling_windows(ForceNM,.5,.1,1000)
    EMGPIwind,EMGPIwindtimes = rolling_windows(PIrms,.5,.1,EMGFs)
    ForcePIwind,ForcePIwindtimes = rolling_windows(ForcePI,.5,.1,1000)
    EMGPMwind,EMGPMwindtimes = rolling_windows(PMrms,.5,.1,EMGFs)
    ForcePMwind,ForcePMwindtimes = rolling_windows(ForcePM,.5,.1,1000)
    EMGSIwind,EMGSIwindtimes = rolling_windows(SIrms,.5,.1,EMGFs)
    ForceSIwind,ForceSIwindtimes = rolling_windows(ForceSI,.5,.1,1000)
    EMGSMwind,EMGSMwindtimes = rolling_windows(SMrms,.5,.1,EMGFs)
    ForceSMwind,ForceSMwindtimes = rolling_windows(ForceSM,.5,.1,1000)
    #should be same # windows for all of them, only need this once
    crossvalsectind = crossvalidataprep(EMGNIwindtimes)

    NIfeats=extrafeatures(EMGNIwind)
    NMfeats=extrafeatures(EMGNMwind)
    #NRfeats=extrafeatures(EMGNRwind)
    #NPfeats=extrafeatures(EMGNPwind)

    PIfeats=extrafeatures(EMGPIwind)
    PMfeats=extrafeatures(EMGPMwind)
    #PRfeats=extrafeatures(EMGPRwind)
    #PPfeats=extrafeatures(EMGPPwind)

    SIfeats=extrafeatures(EMGSIwind)
    SMfeats=extrafeatures(EMGSMwind)
    #SRfeats=extrafeatures(EMGSRwind)
    #SPfeats=extrafeatures(EMGSPwind)

    """Cross-Correlate EMG/SpikeTrains & Force Data to account for delay"""


    #rms
    EMGNIrms=[np.mean(rms(np.expand_dims(window,1),1)) for window in EMGNIwind]
    EMGNMrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGNMwind]
    #EMGNRrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGNRwind]
    #EMGNPrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGNPwind]
    #EMGNIrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGNIwind]

    EMGPIrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGPIwind]
    EMGPMrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGPMwind]
    #EMGPRrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGPRwind]
    #EMGPPrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGPPwind]

    EMGSIrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGSIwind]
    EMGSMrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGSMwind]
    #EMGSRrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGSRwind]
    #EMGSPrms=[np.mean(rms(np.expand_dims(window,0),1)) for window in EMGSPwind]

    # get mean for force windows
    ForceNImean=[np.mean(window,axis=1) for window in ForceNIwind]
    ForceNMmean=[np.mean(window,axis=1) for window in ForceNMwind]
    #ForceNRmean=[np.mean(window,axis=1) for window in ForceNRwind]
    #ForceNPmean=[np.mean(window,axis=1) for window in ForceNPwind]

    ForcePImean=[np.mean(window,axis=1) for window in ForcePIwind]
    ForcePMmean=[np.mean(window,axis=1) for window in ForcePMwind]
    #ForcePRmean=[np.mean(window,axis=1) for window in ForcePRwind]
    #ForcePPmean=[np.mean(window,axis=1) for window in ForcePPwind]

    ForceSImean=[np.mean(window,axis=1) for window in ForceSIwind]
    ForceSMmean=[np.mean(window,axis=1) for window in ForceSMwind]
    #ForceSRmean=[np.mean(window,axis=1) for window in ForceSRwind]
    #ForceSPmean=[np.mean(window,axis=1) for window in ForceSPwind]

    """PLOT: before-after representative cross-correlation to fix lag-time between force & and emg/SpikeTrain data
    Feature Extraction/Engineering:
    """

    #remove extra dim
    #test=np.squeeze(ForceNImean)
    #delay=crosscorr(test,EMGNIrms[0:1605])
    #delay

    #let's do cross correlation
    #tfrm_pipeline=Pipeline([('minmax_scaler', MinMaxScaler())])
    #fi=tfrm_pipeline.fit_transform(np.array(np.squeeze(ForceNImean)).reshape(-1,1))
    #ni=tfrm_pipeline.fit_transform(np.array(EMGNIrms).reshape(-1,1))

    #ftime=np.linspace(0,161,len(test))
    #etime=np.linspace(0,161.11,len(EMGNIrms))
    #plt.plot(ftime,fi,color='r')
    #plt.plot(etime,ni,color='b')

    #NIrms_shifted=np.roll(EMGNIrms,shift=-delay)
    #nis=tfrm_pipeline.fit_transform(NIrms_shifted.reshape(-1,1))
    #plt.plot(etime,nis,color='g')
    #plt.xlim(0,20)

    # get cross correlations for all above
    delayNI=crosscorr(np.squeeze(ForceNImean), EMGNIrms)
    delayNM=crosscorr(np.squeeze(ForceNImean), EMGNMrms)
    #delayNR=crosscorr(np.squeeze(ForceNImean), EMGNRrms)
    #delayNP=crosscorr(np.squeeze(ForceNImean), EMGNPrms)

    delayPI=crosscorr(np.squeeze(ForcePImean), EMGPIrms)
    delayPM=crosscorr(np.squeeze(ForcePMmean), EMGPMrms)
    #delayPR=crosscorr(np.squeeze(ForcePRmean), EMGPRrms)
    #delayPP=crosscorr(np.squeeze(ForcePPmean), EMGPPrms)

    delaySI=crosscorr(np.squeeze(ForceSImean), EMGSIrms)
    delaySM=crosscorr(np.squeeze(ForceSMmean), EMGSMrms)
    #delaySR=crosscorr(np.squeeze(ForceSRmean), EMGSRrms)
    #delaySP=crosscorr(np.squeeze(ForceSPmean), EMGSPrms)

    #Trim the EMG-Time-Domain Features

    smallestLength=1602
    EMGNItrim,ForceNItrim=trim_delay(delayNI,NIfeats,EMGNIrms,ForceNImean,smallestLength)
    print(EMGNItrim)
    EMGNMtrim,ForceNMtrim=trim_delay(delayNM,NMfeats,EMGNMrms,ForceNMmean,smallestLength)
    #EMGNRtrim,ForceNRtrim=trim_delay(delayNR,NRfeats,EMGNRrms,ForceNRmean,smallestLength)
    #EMGNPtrim,ForceNPtrim=trim_delay(delayNP,NPfeats,EMGNPrms,ForceNPmean,smallestLength)

    EMGPItrim,ForcePItrim=trim_delay(delayPI,PIfeats,EMGPIrms,ForcePImean,smallestLength)
    EMGPMtrim,ForcePMtrim=trim_delay(delayPM,PMfeats,EMGPMrms,ForcePMmean,smallestLength)
    #EMGPRtrim,ForcePRtrim=trim_delay(delayPR,PRfeats,EMGPRrms,ForcePRmean,smallestLength)
    #EMGPPtrim,ForcePPtrim=trim_delay(delayPP,PPfeats,EMGPPrms,ForcePPmean,smallestLength)

    EMGSItrim,ForceSItrim=trim_delay(delaySI,SIfeats,EMGSIrms,ForceSImean,smallestLength)
    EMGSMtrim,ForceSMtrim=trim_delay(delaySM,SMfeats,EMGSMrms,ForceSMmean,smallestLength)
    #EMGSRtrim,ForceSRtrim=trim_delay(delaySR,SRfeats,EMGSRrms,ForceSRmean,smallestLength)
    #EMGSPtrim,ForceSPtrim=trim_delay(delaySP,SPfeats,EMGSPrms,ForceSPmean,smallestLength)

    #get cross-correlation for MUs
    MUdelayNNI = crosscorr(np.squeeze(ForceNImean), FRwindNNI)
    MUdelayNNM = crosscorr(np.squeeze(ForceNMmean), FRwindNNM)
    #MUdelayNNR = crosscorr(np.squeeze(ForceNRmean), FRwindNNR)
    #MUdelayNNP = crosscorr(np.squeeze(ForceNPmean), FRwindNNP)

    MUdelayNPI = crosscorr(np.squeeze(ForcePImean), FRwindNPI)
    MUdelayNPM = crosscorr(np.squeeze(ForcePMmean), FRwindNPM)
    #MUdelayNPR = crosscorr(np.squeeze(ForcePRmean), FRwindNPR)
    #MUdelayNPP = crosscorr(np.squeeze(ForcePPmean), FRwindNPP)

    MUdelayNSI = crosscorr(np.squeeze(ForceSImean), FRwindNSI)
    MUdelayNSM = crosscorr(np.squeeze(ForceSMmean), FRwindNSM)
    #MUdelayNSR = crosscorr(np.squeeze(ForceSRmean), FRwindNSR)
    #MUdelayNSP = crosscorr(np.squeeze(ForceSPmean), FRwindNSP)

    MUdelayPPI = crosscorr(np.squeeze(ForcePImean), FRwindPPI)
    MUdelayPPM = crosscorr(np.squeeze(ForcePMmean), FRwindPPM)
    #MUdelayPPR = crosscorr(np.squeeze(ForcePRmean), FRwindPPR)
    #MUdelayPPP = crosscorr(np.squeeze(ForcePPmean), FRwindPPP)

    MUdelaySSI = crosscorr(np.squeeze(ForceSImean), FRwindSSI)
    MUdelaySSM = crosscorr(np.squeeze(ForceSMmean), FRwindSSM)
    #MUdelaySSR = crosscorr(np.squeeze(ForceSRmean), FRwindSSR)
    #MUdelaySSP = crosscorr(np.squeeze(ForceSPmean), FRwindSSP)

    #cross-correlate MU spike train and force data then shift and cut based on delay
    
    #trim Firing rates to be same length as force after cross-correlation

    #MU inputs
    smallestLength = 1590
    FRNNItrim,SpkFrcNNI = trim_MUdelay(MUdelayNNI,FRwindNNI,ForceNImean,smallestLength)
    FRNNMtrim,SpkFrcNNM = trim_MUdelay(MUdelayNNM,FRwindNNM,ForceNMmean,smallestLength)
    #FRNNRtrim,SpkFrcNNR = trim_MUdelay(MUdelayNNR,FRwindNNR,ForceNRmean,smallestLength)
    #FRNNPtrim,SpkFrcNNP = trim_MUdelay(MUdelayNNP,FRwindNNP,ForceNPmean,smallestLength)

    FRNPItrim,SpkFrcNPI = trim_MUdelay(MUdelayNPI,FRwindNPI,ForcePImean,smallestLength)
    FRNPMtrim,SpkFrcNPM = trim_MUdelay(MUdelayNPM,FRwindNPM,ForcePMmean,smallestLength)
    #FRNPRtrim,SpkFrcNPR = trim_MUdelay(MUdelayNPR,FRwindNPR,ForcePRmean,smallestLength)
    #FRNPPtrim,SpkFrcNPP = trim_MUdelay(MUdelayNPP,FRwindNPP,ForcePPmean,smallestLength)

    FRNSItrim,SpkFrcNSI = trim_MUdelay(MUdelayNSI,FRwindNSI,ForceSImean,smallestLength)
    FRNSMtrim,SpkFrcNSM = trim_MUdelay(MUdelayNSM,FRwindNSM,ForceSMmean,smallestLength)
    #FRNSRtrim,SpkFrcNSR = trim_MUdelay(MUdelayNSR,FRwindNSR,ForceSRmean,smallestLength)
    #FRNSPtrim,SpkFrcNSP = trim_MUdelay(MUdelayNSP,FRwindNSP,ForceSPmean,smallestLength)

    FRPPItrim,SpkFrcPPI = trim_MUdelay(MUdelayPPI,FRwindPPI,ForcePImean,smallestLength)
    FRPPMtrim,SpkFrcPPM = trim_MUdelay(MUdelayPPM,FRwindPPM,ForcePMmean,smallestLength)
    #FRPPRtrim,SpkFrcPPR = trim_MUdelay(MUdelayPPR,FRwindPPR,ForcePRmean,smallestLength)
    #FRPPPtrim,SpkFrcPPP = trim_MUdelay(MUdelayPPP,FRwindPPP,ForcePPmean,smallestLength)

    FRSSItrim,SpkFrcSSI = trim_MUdelay(MUdelaySSI,FRwindSSI,ForceSImean,smallestLength)
    FRSSMtrim,SpkFrcSSM = trim_MUdelay(MUdelaySSM,FRwindSSM,ForceSMmean,smallestLength)
    #FRSSRtrim,SpkFrcSSR = trim_MUdelay(MUdelaySSR,FRwindSSR,ForceSRmean,smallestLength)
    #FRSSPtrim,SpkFrcSSP = trim_MUdelay(MUdelaySSP,FRwindSSP,ForceSPmean,smallestLength)

    #np.shape(SpkFrcSSI)

    """### Prep Input-Outputs"""
    ind = np.zeros((EMGNItrim.shape[0], 1))
    mid = np.ones((EMGNMtrim.shape[0], 1))
    Finger = np.concatenate((ind,mid),axis=0)
    #EMG Time-Domain Features
    #combine all fingers for one model (treat fingers as features)
    X_EMGNeu_p = np.concatenate((EMGNItrim,EMGNMtrim),axis = 0)
    X_EMGNeu = np.concatenate((X_EMGNeu_p,Finger),axis = 1)
    #X_EMGNeu = np.concatenate((X_EMGNeu,EMGNRtrim),axis = 1)
    #X_EMGNeu = np.concatenate((X_EMGNeu,EMGNPtrim),axis = 1)

    X_EMGPro_p = np.concatenate((EMGPItrim,EMGPMtrim),axis = 0)
    X_EMGPro = np.concatenate((X_EMGPro_p,Finger),axis = 1)
    #X_EMGPro = np.concatenate((X_EMGPro,EMGPRtrim),axis = 1)
    #X_EMGPro = np.concatenate((X_EMGPro,EMGPPtrim),axis = 1)

    X_EMGSup_p = np.concatenate((EMGSItrim,EMGSMtrim),axis = 0)
    X_EMGSup = np.concatenate((X_EMGSup_p,Finger),axis = 1)
    #X_EMGSup = np.concatenate((X_EMGSup,EMGSRtrim),axis = 1)
    #X_EMGSup = np.concatenate((X_EMGSup,EMGSPtrim),axis = 1)

    #PCA Components
    #Index

    #Middle

    #Ring

    #Pinky

    #combine all fingers for one model (treat fingers as features)

    #MUs
    FRNNI = np.reshape(FRNNItrim,[-1,1])
    FRNNM = np.reshape(FRNNMtrim,[-1,1])
    #FRNNR = np.reshape(FRNNRtrim,[-1,1])
    #FRNNP = np.reshape(FRNNPtrim,[-1,1])

    FRNPI = np.reshape(FRNPItrim,[-1,1])
    FRNPM = np.reshape(FRNPMtrim,[-1,1])
    #FRNPR = np.reshape(FRNPRtrim,[-1,1])
    #FRNPP = np.reshape(FRNPPtrim,[-1,1])

    FRNSI = np.reshape(FRNSItrim,[-1,1])
    FRNSM = np.reshape(FRNSMtrim,[-1,1])
    #FRNSR = np.reshape(FRNSRtrim,[-1,1])
    #FRNSP = np.reshape(FRNSPtrim,[-1,1])

    FRPPI = np.reshape(FRPPItrim,[-1,1])
    FRPPM = np.reshape(FRPPMtrim,[-1,1])
    #FRPPR = np.reshape(FRPPRtrim,[-1,1])
    #FRPPP = np.reshape(FRPPPtrim,[-1,1])

    FRSSI = np.reshape(FRSSItrim,[-1,1])
    FRSSM = np.reshape(FRSSMtrim,[-1,1])
    #FRSSR = np.reshape(FRSSRtrim,[-1,1])
    #FRSSP = np.reshape(FRSSPtrim,[-1,1])

    #index = zeros
    #middle = ones
    #encode for Index & Middle fingers together for spiketrains
    #combine index/middle labels/forces



    FRNNeu = np.concatenate((FRNNI,FRNNM))
    indtmp = np.zeros([len(FRNNI),1])
    midtmp = np.ones([len(FRNNM),1])
    ohefing = np.concatenate((indtmp,midtmp))
    X_FRNNeu = np.concatenate((FRNNeu,ohefing),axis=1)
    y_SpksFrcNNeu = np.concatenate((SpkFrcNNI,SpkFrcNNM)).reshape(-1,1)
    #
    FRNPro = np.concatenate((FRNPI,FRNPM))
    indtmp = np.zeros([len(FRNPI),1])
    midtmp = np.ones([len(FRNPM),1])
    ohefing = np.concatenate((indtmp,midtmp))
    X_FRNPro = np.concatenate((FRNPro,ohefing),axis=1)
    y_SpksFrcNPro = np.concatenate((SpkFrcNPI,SpkFrcNPM)).reshape(-1,1)
    #
    FRNSup = np.concatenate((FRNSI,FRNSM))
    indtmp = np.zeros([len(FRNSI),1])
    midtmp = np.ones([len(FRNSM),1])
    ohefing = np.concatenate((indtmp,midtmp))
    X_FRNSup = np.concatenate((FRNSup,ohefing),axis=1)
    y_SpksFrcNSup = np.concatenate((SpkFrcNSI,SpkFrcNSM)).reshape(-1,1)
    #
    FRPPro = np.concatenate((FRPPI,FRPPM))
    indtmp = np.zeros([len(FRPPI),1])
    midtmp = np.ones([len(FRPPM),1])
    ohefing = np.concatenate((indtmp,midtmp))
    X_FRPPro = np.concatenate((FRPPro,ohefing),axis=1)
    y_SpksFrcPPro = np.concatenate((SpkFrcPPI,SpkFrcPPM)).reshape(-1,1)
    #
    FRSSup = np.concatenate((FRSSI,FRSSM))
    indtmp = np.zeros([len(FRSSI),1])
    midtmp = np.ones([len(FRSSM),1])
    ohefing = np.concatenate((indtmp,midtmp))
    X_FRSSup = np.concatenate((FRSSup,ohefing),axis=1)
    y_SpksFrcSSup = np.concatenate((SpkFrcSSI,SpkFrcSSM)).reshape(-1,1)

    # X_FRNNeu
    # X_FRNPro
    # X_FRNSup
    # X_FRPPro
    # X_FRSSup
    # y_SpksFrcNNeu
    # y_SpksFrcNPro
    # y_SpksFrcNSup
    # y_SpksFrcPPro
    # y_SpksFrcSSup

    #Force
    #reshape vectors to concatenate
    FrcNI = np.reshape(ForceNItrim,[-1,1])
    FrcNM = np.reshape(ForceNMtrim,[-1,1])
    #FrcNR = np.reshape(ForceNRtrim,[-1,1])
    #FrcNP = np.reshape(ForceNPtrim,[-1,1])

    FrcPI = np.reshape(ForceSItrim,[-1,1])
    FrcPM = np.reshape(ForceSMtrim,[-1,1])
    #FrcPR = np.reshape(ForceSRtrim,[-1,1])
    #FrcPP = np.reshape(ForceSPtrim,[-1,1])

    FrcSI = np.reshape(ForceSItrim,[-1,1])
    FrcSM = np.reshape(ForceSMtrim,[-1,1])
    #FrcSR = np.reshape(ForceSRtrim,[-1,1])
    #FrcSP = np.reshape(ForceSPtrim,[-1,1])

    #combine all fingers for one model (treat fingers as features)
    y_Neu = np.concatenate((FrcNI,FrcNM),axis=0)
    #y_Neu = np.concatenate((y_Neu,FrcNR),axis=1)
    #y_Neu = np.concatenate((y_Neu,FrcNP),axis=1)

    y_Pro = np.concatenate((FrcPI,FrcPM),axis=0)
    #y_Pro = np.concatenate((y_Pro,FrcPR),axis=1)
    #y_Pro = np.concatenate((y_Pro,FrcPP),axis=1)

    y_Sup = np.concatenate((FrcSI,FrcSM),axis=0)
    #y_Sup = np.concatenate((y_Sup,FrcSR),axis=1)
    #y_Sup = np.concatenate((y_Sup,FrcSP),axis=1)

    #Sang
    # Models to test:
    # X_EMGNeu, y_Neu
    # X_EMGNeu, y_Pro
    # X_EMGNeu, y_Sup
    # X_EMGPro, y_Pro
    # X_EMGSup, y_Sup

    return X_EMGNeu,X_EMGPro,X_EMGSup,y_Neu,y_Pro,y_Sup,X_FRNNeu,X_FRNPro,X_FRNSup, X_FRPPro, X_FRSSup, y_SpksFrcNNeu, y_SpksFrcNPro,y_SpksFrcNSup,y_SpksFrcPPro,y_SpksFrcSSup

In [6]:
allfeatsneu=pd.DataFrame(columns=['Subject','zc','maxamp','rms','Finger','y_Neu'])
allfeatspro=pd.DataFrame(columns=['Subject','zc','maxamp','rms','Finger','y_Pro'])
allfeatssup=pd.DataFrame(columns=['Subject','zc','maxamp','rms','Finger','y_Sup'])
allneusparks=pd.DataFrame(columns=['Subject','Finger','X_FRNNeu','y_SpksFrcNNeu'])
allprosparks=pd.DataFrame(columns=['Subject','Finger','X_FRNPro','X_FRPPro','y_SpksFrcNPro','y_SpksFrcPPro'])
allsupsparks=pd.DataFrame(columns=['Subject','Finger','X_FRNSup','X_FRSSup','y_SpksFrcNSup','y_SpksFrcSSup'])
for subject in Subjects:
  neu=pd.DataFrame(columns=['Subject','zc','maxamp','rms','Finger','y_Neu'])
  neusparks=pd.DataFrame(columns=['Subject','Finger','X_FRNNeu','y_SpksFrcNNeu'])
  pro=pd.DataFrame(columns=['Subject','zc','maxamp','rms','Finger','y_Pro'])
  prosparks=pd.DataFrame(columns=['Subject','Finger','X_FRNPro','X_FRPPro','y_SpksFrcNPro','y_SpksFrcPPro'])
  sup=pd.DataFrame(columns=['Subject','zc','maxamp','rms','Finger','y_Sup'])
  supsparks=pd.DataFrame(columns=['Subject','Finger','X_FRNSup','X_FRSSup','y_SpksFrcNSup','y_SpksFrcSSup'])
  X_EMGNeu,X_EMGPro,X_EMGSup,y_Neu,y_Pro,y_Sup,X_FRNNeu,X_FRNPro,X_FRNSup, X_FRPPro, X_FRSSup, y_SpksFrcNNeu, y_SpksFrcNPro,y_SpksFrcNSup,y_SpksFrcPPro,y_SpksFrcSSup=get_subj_var(Force_dir,EMG_dir,Spks_dir,subject)
  #neu['Subject']=pd.Series([subject]*len(X_EMGNeu[:,0]))
  #neu['zc']=X_EMGNeu[:,0]
  #neu['maxamp']=X_EMGNeu[:,1]
  #neu['rms']=X_EMGNeu[:,2]
  #neu['Finger']=X_EMGNeu[:,3]
  #neu['y_Neu']=y_Neu
  size=min(len(X_FRNNeu[:,0]),len(y_SpksFrcNNeu),len(y_SpksFrcNPro),len(y_SpksFrcPPro),len(y_SpksFrcNSup),len(y_SpksFrcSSup))
  neusparks['Subject']=pd.Series([subject]*size)
  neusparks['X_FRNneu']=X_FRNNeu[:,0][0:size]
  neusparks['Finger']=X_FRNNeu[:,1][0:size]
  neusparks['y_SpksFrcNNeu']=y_SpksFrcNNeu[0:size]
  #pro['zc']=X_EMGPro[:,0]
  #pro['maxamp']=X_EMGPro[:,1]
  #pro['rms']=X_EMGPro[:,2]
  #pro['Finger']=X_EMGPro[:,3]
  #pro['y_Pro']=y_Pro
  #pro['Subject']=pd.Series([subject]*len(X_EMGPro[:,0]))
  prosparks['X_FRNPro']=X_FRNPro[:,0][0:size]
  prosparks['Finger']=X_FRNPro[:,1][0:size]
  prosparks['Subject']=pd.Series([subject]*size)
  prosparks['y_SpksFrcNPro']=y_SpksFrcNPro[0:size]
  prosparks['y_SpksFrcPPro']=y_SpksFrcPPro[0:size]  
  #sup['zc']=X_EMGSup[:,0]
  #sup['maxamp']=X_EMGSup[:,1]
  #sup['rms']=X_EMGSup[:,2]
  #sup['Finger']=X_EMGSup[:,3]
  #sup['y_Sup']=y_Sup
  #sup['Subject']=pd.Series([subject]*len(X_EMGSup[:,0]))  
  supsparks['X_FRSSup']=X_FRSSup[:,0][0:size]
  supsparks['Finger']=X_FRSSup[:,1][0:size]
  #supsparks['y_SpksFrcNSup']=y_SpksFrcNSup
  supsparks['y_SpksFrcSSup']=y_SpksFrcSSup[0:size]
  supsparks['Subject']=pd.Series([subject]*size)  
  #neu.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/'+subject+'_neu.csv')
  neusparks.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/'+subject+'_neuspks.csv')
  #pro.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/'+subject+'_pro.csv')
  prosparks.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/'+subject+'_prospks.csv')
  #sup.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/'+subject+'_sup.csv')
  supsparks.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/'+subject+'_supspks.csv')
  #allfeatsneu=pd.concat([allfeatsneu,neu])
  allneusparks=pd.concat([allneusparks,neusparks])
  #allfeatspro=pd.concat([allfeatspro,pro])
  allprosparks=pd.concat([allprosparks,prosparks])
  #allfeatssup=pd.concat([allfeatsneu,sup])
  allsupsparks=pd.concat([allsupsparks,supsparks])

force


OSError: ignored

In [18]:
y_SpksFrcNPro.shape

(3180, 1)

In [0]:
#allfeatsneu.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/allneu.csv')
#allfeatspro.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/allpro.csv')
#allfeatssup.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/allsup.csv')
allneusparks.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/allneuspks.csv')
allprosparks.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/allprospks.csv')
allsupsparks.to_csv('./gdrive/My Drive/BMME890_MachineLearning/ProjectData/allsupspks.csv')